<a href="https://colab.research.google.com/github/greenday1234/AI/blob/main/TensorflowTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### tensorflow import
print 는 굳이 없어도 됨

In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


### mnist 데이터 셋 사용
학습 데이터와 테스트 데이터를 넣음

샘플 데이터를 정수에서 부동 소수점으로 변환

x 는 모델에 입력되는 데이터, y 는 모델이 예측하거나 분류해야 하는 정답 데이터

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 0s 0us/step


### 모델 빌드하기
훈련에 사용할 옵티마이저와 손실 함수 선택

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

softmax 활성화 함수를 사용해 모델을 빌드했기 때문에 확률로 변환된 값이 출력됨

즉 아래의 출력값은 0~9까지의 숫자에 대한 확률임

In [6]:
predictions = model(x_train[:1]).numpy()
predictions

array([[0.08113955, 0.05181451, 0.06074198, 0.11051384, 0.14221302,
        0.10530333, 0.08740725, 0.12551916, 0.14931297, 0.08603442]],
      dtype=float32)

### softmax 활성화 함수 사용

하지만 이 방법은 정확하지 않고, 안정적인 손실 계산이 불가능해 권장 X



In [7]:
tf.nn.softmax(predictions).numpy()

array([[0.09808461, 0.09525004, 0.09610419, 0.1010085 , 0.10426168,
        0.10048357, 0.09870131, 0.10253561, 0.10500457, 0.0985659 ]],
      dtype=float32)

이 손실은 실제 클래스의 음의 로그 확률과 같음

즉 모델이 올바른 클래스를 확신하는 경우 손실은 0임



In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

훈련되지 않았기 때문에 초기 손실은 -tf.math.log(1/10) 이므로 2.3에 근접해야 함

In [9]:
loss_fn(y_train[:1], predictions).numpy()

2.297761

### 훈련 시작 전에 loss 를 변경해 모델 컴파일 진행

In [10]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

### 모델 학습
epochs 는 데이터 셋을 몇 번 반복해 학습할 것인지 나타내는 매개변수임

In [11]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1875/1875 [==============================] - 10s 4ms/step - loss: 0.2964 - accuracy: 0.9134
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1450 - accuracy: 0.9569
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1076 - accuracy: 0.9671
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0868 - accuracy: 0.9729
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0744 - accuracy: 0.9766


evaluate 는 모델의 성능을 평가하는 데 사용

verbose=2 는 각 epoch 마다 훈련과정을 출력함

In [15]:
model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0408 - accuracy: 0.9865
Epoch 2/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0386 - accuracy: 0.9873
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0365 - accuracy: 0.9876
Epoch 4/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0333 - accuracy: 0.9892
Epoch 5/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0322 - accuracy: 0.9891
313/313 - 1s - loss: 0.0759 - accuracy: 0.9811 - 582ms/epoch - 2ms/step


[0.07587451487779617, 0.9811000227928162]

모델이 확률을 반환하게 하려면 해당 모델을 래핑하고 softmax 를 첨부하면 됨

In [13]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [14]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.08533682, 0.08533682, 0.08533682, 0.08533733, 0.08533682,
        0.08533682, 0.08533682, 0.2319681 , 0.08533682, 0.08533682],
       [0.08533675, 0.08533682, 0.2319692 , 0.08533676, 0.08533675,
        0.08533675, 0.08533675, 0.08533675, 0.08533675, 0.08533675],
       [0.08533756, 0.23195629, 0.08533935, 0.08533758, 0.08533766,
        0.08533757, 0.08533758, 0.08533962, 0.0853392 , 0.08533756],
       [0.23194528, 0.08533826, 0.08534851, 0.08533826, 0.08533829,
        0.08533826, 0.08533829, 0.08533833, 0.08533826, 0.08533827],
       [0.08533712, 0.08533712, 0.08533712, 0.08533712, 0.23196347,
        0.08533712, 0.08533712, 0.08533714, 0.08533712, 0.08533961]],
      dtype=float32)>